# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [1]:
from typing import Optional #used to specify that a function parameter or return value can be of a given type or None

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

FAISS: crucial for BabyAGI because they allow it to effectively store and retrieve embeddings that it generates during task execution. For instance, BabyAGI might need to find related tasks, store important information, or recall past interactions.

In [2]:
#%pip install faiss-cpu #ALLOWS BABYAGI TO STORE AND RETRIEVE EMBEDDINGS THAT IT GENERATES DURING TASK EXECUTION

Google-search-results: used to provide the BabyAGI agent with access to up-to-date, real-world information.

In [3]:
#%pip install google-search-results #TO INTERACT WITH SERPAPI

related to connecting BabyAGI with a vector store, which is used for storing vector representations of text data.

In [4]:
# %pip install faiss-cpu > /dev/null
# %pip install google-search-results > /dev/null
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

 for managing environment variables, specifically to securely load API keys needed for the lab.

In [5]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

 setting up a vector store, which is essential for efficiently storing and retrieving vector embeddings, allowing BabyAGI to work with contextual or related information.

In [6]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


## Define the Chains

BabyAGI relies on three LLM chains:
- **Task creation chain to select new tasks to add to the list:** 
  The task creation chain is responsible for generating new tasks based on the output of previously completed tasks.  The chain uses a Large Language Model (LLM) to analyze the current results and decide what additional tasks might be helpful to further progress.


- **Task prioritization chain to re-prioritize tasks:** 
  This chain is used to re-prioritize tasks dynamically. The tasks in the list may not all have the same level of importance or urgency, and their priorities may shift as new information becomes available. Using an LLM, this chain re-evaluates the list of tasks and orders them according to the current context and the overall objective. This helps ensure that BabyAGI always focuses on the most important or impactful tasks first,


- **Execution Chain to execute the tasks:** 
  carrying out the tasks on the task list. nvolves interacting with the LLM to solve problems, gather information, or generate outputs based on the task requirements. The execution chain in this notebook will now be enhanced into an agent. This means that, rather than relying solely on the LLM to "make up" answers or generate responses, it will make use of tools and APIs to gather real data. access up-to-date, verifiable information and execute more sophisticated tasks.


NOTE: in this notebook, the Execution chain will now be an agent.

In [7]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="Create todo list",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

C:\Users\fmrol\AppData\Local\Temp\ipykernel_26872\2860866751.py:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)


In [8]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

C:\Users\fmrol\AppData\Local\Temp\ipykernel_26872\3064747660.py:4: LangChainDeprecationWarning: The class `ZeroShotAgent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~create_react_agent` instead.
  agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)


### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [9]:
OBJECTIVE = "Write a report on the political crisis in Venezuela during the last month"

In [10]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [11]:
baby_agi({"objective": OBJECTIVE})

C:\Users\fmrol\AppData\Local\Temp\ipykernel_26872\3867971396.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  baby_agi({"objective": OBJECTIVE})



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I should create a todo list based on the objective given
Action: Create todo list
Action Input: "Write a report on the political crisis in Venezuela during the last month."
Observation: 

1. Research the current political situation in Venezuela, including recent events and key players involved.
2. Gather reliable sources and information from reputable news outlets, government websites, and academic sources.
3. Create an outline for the report, including an introduction, background information, key events, and analysis.
4. Begin writing the report, starting with the introduction and background information.
5. Include relevant statistics and data to support your analysis.
6. Use quotes and interviews from experts or individuals directly involved in the crisis to provide insight and perspective.
7. Address any potential biases or conflicting information in y

{'objective': 'Write a report on the political crisis in Venezuela during the last month'}